In [ ]:
# Built-in imports
import os
import shutil
import warnings

# Third-part imports
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# My imports
from functions import *

In [ ]:
warnings.filterwarnings('ignore')

device: str = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'

x_img_resize: int = 320
y_img_resize: int = 320
img_resize: tuple = (x_img_resize, y_img_resize)

x_figure_size: int = 16
y_figure_size: int = 8
fig_size: tuple = (x_figure_size, y_figure_size)

parametri_grafici = {
    'figure.figsize': fig_size, # Dimensione della figura
    'figure.autolayout': True,  # Regolazione automatica delle dimensioni della figura
    'figure.titlesize': 20,     # Dimensione del titolo associato ad ogni figura (plt.suptitle())
    'axes.titlesize': 20,       # Dimensione del titolo associato ad ogni grafico all'interno di una figura (plt.title())
    'axes.labelsize': 20,       # Dimensione delle etichette sia sull'asse x sia sull'asse y
    'xtick.labelsize': 15,      # Dimensione dei riferimenti sull'asse x
    'ytick.labelsize': 15,      # Dimensione dei riferimenti sull'asse y
}
plt.rcParams.update(parametri_grafici)

flag_new_images: bool = False          # True solo quando vengono aggiunte nuove immagini al path.
flag_preds_display: bool = False        # True per visualizzare le predizioni su ogni immagine del dataset.
flag_gradcam_display: bool = False      # True per visualizzare le immagini con le RoI usate dal modello per fare la predizione.
flag_performance_display: bool = False  # True solo quando si vogliono visualizzare le performance della rete al variare di epsilon.

In [ ]:
path_cartella_immagini: str = '../ResNet_FGSM/images'
estensione: str = '.jpg'

if flag_new_images:

    # Per comodità rinomino tutte le immagini su cui fare inferenza con nomi del tipo 1.jpg, 2.jpg, 3.jpg, ...
    files: list = [os.path.join(path_cartella_immagini, elem) for elem in os.listdir(path_cartella_immagini) if not elem.startswith('.')]
    num: int = 1
    for file in files:
        os.rename(file, str(num) + estensione)
        num = num + 1

    # Poichè a seguito della ridenominazione le immagini vengono spostate al di fuori dalla cartella images, le riporto dentro.
    temp_path: str = '../ResNet_FGSM'
    files: list = [os.path.join(temp_path, elem) for elem in os.listdir(temp_path) if not elem.startswith('.') and elem.endswith(estensione)]
    for file in files:
        shutil.move(os.path.join(temp_path, file), path_cartella_immagini)

# Creo il dataset con tutte le immagini su cui eseguire l'attacco FGSM.
files: list = [os.path.join(path_cartella_immagini, elem) for elem in os.listdir(path_cartella_immagini) if not elem.startswith('.')]
dataset = list()
for file in files:
    dataset.append(file)

# Opzionale: ordino le immagini del dataset in base al nome.
dataset = sorted(dataset, key = lambda x: int(x[x.rfind('/') + 1 : x.rfind('.')]))

# Definizione del modello di classificazione ResNet trainato sul dataset ImageNet.
model = resnet50(weights = ResNet50_Weights.IMAGENET1K_V2).to(device) # Lista delle classi: https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/
model.eval()

# Eseguo l'attacco FGSM sulle immagini del dataset sfruttando un valore di epsilon scelto arbitrariamente.
triplas: list = []
for image in dataset:
    original_image: torch.Tensor = read_image(image)
    original_image: torch.Tensor = preprocess(original_image, img_resize).to(device)
    epsilon: float = 0.8
    perturbed_image, noise = fgsm_attack(model, original_image, epsilon, device)
    original_image: torch.Tensor = postprocess(original_image)
    perturbed_image: torch.Tensor = postprocess(perturbed_image)
    tripla: tuple = (original_image, noise, perturbed_image)
    triplas.append(tripla)

In [ ]:
if flag_preds_display:
    for tripla in triplas:
        preds_display(model, tripla, epsilon, show_noise = True)

In [ ]:
if flag_gradcam_display:
    for tripla in triplas:
        gradcam_display(model, tripla, img_resize)

In [ ]:
if flag_performance_display:
    epsilons: list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    performance_display(dataset, model, img_resize, device, epsilons, show_wrong_preds = True)